In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()

In [3]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

X = minmax.fit_transform(cancer['data'])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, cancer['target'], test_size=0.3, random_state=1)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((398, 30), (171, 30), (398,), (171,))

In [9]:
print(f"스케일 조정 전 특성별 최솟값: \n{cancer['data'].min(axis=0)}")
print(f"스케일 조정 후 특성별 최솟값: \n{X.min(axis=0)}")
print('='*100)
print(f"스케일 조정 전 특성별 최댓값: \n{cancer['data'].max(axis=0)}")
print(f"스케일 조정 후 특성별 최댓값: \n{X.max(axis=0)}")

스케일 조정 전 특성별 최솟값: 
[6.981e+00 9.710e+00 4.379e+01 1.435e+02 5.263e-02 1.938e-02 0.000e+00
 0.000e+00 1.060e-01 4.996e-02 1.115e-01 3.602e-01 7.570e-01 6.802e+00
 1.713e-03 2.252e-03 0.000e+00 0.000e+00 7.882e-03 8.948e-04 7.930e+00
 1.202e+01 5.041e+01 1.852e+02 7.117e-02 2.729e-02 0.000e+00 0.000e+00
 1.565e-01 5.504e-02]
스케일 조정 후 특성별 최솟값: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
스케일 조정 전 특성별 최댓값: 
[2.811e+01 3.928e+01 1.885e+02 2.501e+03 1.634e-01 3.454e-01 4.268e-01
 2.012e-01 3.040e-01 9.744e-02 2.873e+00 4.885e+00 2.198e+01 5.422e+02
 3.113e-02 1.354e-01 3.960e-01 5.279e-02 7.895e-02 2.984e-02 3.604e+01
 4.954e+01 2.512e+02 4.254e+03 2.226e-01 1.058e+00 1.252e+00 2.910e-01
 6.638e-01 2.075e-01]
스케일 조정 후 특성별 최댓값: 
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


In [10]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_std = std.fit_transform(cancer['data'])

In [12]:
print(f"Standard Scaler 적용 전: \n{cancer['data'][:1, :]}")
print(f"Standard Scaler 적용 후: \n{X_std[:1, :]}")

Standard Scaler 적용 전: 
[[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
  1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
  6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
  1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
  4.601e-01 1.189e-01]]
Standard Scaler 적용 후: 
[[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
   2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
   2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
   1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
   1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]]


- KNN
- 선형회귀 + 규제회귀
- 로지스틱 회귀
- 서포트 벡터 머신

- 의사결정나무
- 랜덤 포레스트
- XGBoost
- LightGBM